In [9]:
import cv2
import numpy as np

In [13]:
cap = cv2.VideoCapture(1)

imgTarget = cv2.imread('TargetImage.jpg')
myVid = cv2.VideoCapture('video.mp4')

detection = False
frameCounter = 0

# The first frame of the video
success, imgVideo = myVid.read()

# Resize the target and video to the same size
hT, wT, cT = imgTarget.shape
imgVideo =cv2.resize(imgVideo, (wT, hT))

# Object detector
orb = cv2.ORB_create(nfeatures=1000)
#key point and descriptor
kp1, des1 = orb.detectAndCompute(imgTarget, None)
#imgTarget = cv2.drawKeypoints(imgTarget, kp1, None)


def stackImages(imgArray,scale,lables=[]):
    sizeW= imgArray[0][0].shape[1]
    sizeH = imgArray[0][0].shape[0]
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                imgArray[x][y] = cv2.resize(imgArray[x][y], (sizeW,sizeH), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((sizeH, sizeW, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
            hor_con[x] = np.concatenate(imgArray[x])
        ver = np.vstack(hor)
        ver_con = np.concatenate(hor)
    else:
        for x in range(0, rows):
            imgArray[x] = cv2.resize(imgArray[x], (sizeW, sizeH), None, scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        hor_con= np.concatenate(imgArray)
        ver = hor
    if len(lables) != 0:
        eachImgWidth= int(ver.shape[1] / cols)
        eachImgHeight = int(ver.shape[0] / rows)
        print(eachImgHeight)
        for d in range(0, rows):
            for c in range (0,cols):
                cv2.rectangle(ver,(c*eachImgWidth,eachImgHeight*d),(c*eachImgWidth+len(lables[d])*13+27,30+eachImgHeight*d),(255,255,255),cv2.FILLED)
                cv2.putText(ver,lables[d],(eachImgWidth*c+10,eachImgHeight*d+20),cv2.FONT_HERSHEY_COMPLEX,0.7,(255,0,255),2)
    return ver


while True:
    ret, imgWebcam = cap.read()
    imgAug = imgWebcam.copy()
    
    # find descriptors as keypoints for our live video
    kp2, des2 = orb.detectAndCompute(imgWebcam, None)
    #imgWebcam = cv2.drawKeypoints(imgWebcam, kp2, None)
    
    if detection == False:
        myVid.set(cv2.CAP_PROP_POS_FRAMES, 0)
        frameCounter = 0
    else:
        if frameCounter == myVid.get(cv2.CAP_PROP_FRAME_COUNT):
            myVid.set(cv2.CAP_PROP_POS_FRAMES, 0)
            frameCounter = 0
        success, imgVideo = myVid.read()
        imgVideo =cv2.resize(imgVideo, (wT, hT))
        
    
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)
    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good.append(m)
    print(len(good))
    
    imgFeatures = cv2.drawMatches(imgTarget, kp1, imgWebcam, kp2, good, None, flags=2)
    
    if len(good) > 20:
        detection = True
        
        srcPts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
        dstPts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
        
        matrix, mask = cv2.findHomography(srcPts, dstPts, cv2.RANSAC, 5)
        print(matrix)
        
        pts = np.float32([[0, 0], [0, hT], [wT, hT], [wT, 0]]).reshape(-1, 1, 2)
        dst = cv2.perspectiveTransform(pts, matrix)
        img2 = cv2.polylines(imgWebcam, [np.int32(dst)], True, (255, 0, 255), 3)
        
        imgWarp = cv2.warpPerspective(imgVideo, matrix, (imgWebcam.shape[1], imgWebcam.shape[0]))
        
        maskNew = np.zeros((imgWebcam.shape[0], imgWebcam.shape[1]), np.uint8)
        cv2.fillPoly(maskNew, [np.int32(dst)], (255, 255, 255))
        maskInv = cv2.bitwise_not(maskNew)
        imgAug = cv2.bitwise_and(imgAug, imgAug, mask = maskInv)
        imgAug = cv2.bitwise_or(imgWarp, imgAug)
        
        imgStacked = stackImages(([imgWebcam, imgVideo, imgTarget], [imgFeatures, imgWarp, imgAug]), 0.5)
       
        cv2.imshow('imgStacked', imgStacked)
    
    '''
        cv2.imshow('maskInv', imgAug) 
        cv2.imshow('img2', img2) 
        cv2.imshow('imgWarp', imgWarp)
        
            
    cv2.imshow('imgFeatures', imgFeatures)
    cv2.imshow('ImgTarget', imgTarget)
    cv2.imshow('myVid', imgVideo)
    cv2.imshow('Webcam', imgWebcam)
    '''
    frameCounter += 1
    
    k=cv2.waitKey(1)
    if k == 27:
        #通过esc键退出摄像
        cv2.destroyAllWindows()
        break
#关闭摄像头
cap.release()

11
4
6
9
10
10
9
8
8
9
8
6
7
5
7
2
5
8
4
7
9
5
10
6
32
[[ 3.36538955e-01 -2.86258950e-02  1.90539923e+02]
 [ 1.34865176e-02  3.17517263e-01  1.46298415e+02]
 [ 7.79442827e-06 -4.67924819e-05  1.00000000e+00]]
39
[[ 3.43253870e-01 -1.81770693e-02  1.76159199e+02]
 [ 9.80314686e-03  3.34341831e-01  1.38226548e+02]
 [ 2.56299995e-06 -2.06941869e-05  1.00000000e+00]]
38
[[ 4.06525081e-01 -2.74525567e-02  1.18406029e+02]
 [ 3.19599214e-02  3.87270086e-01  1.14668024e+02]
 [ 4.71803597e-05 -1.71884191e-05  1.00000000e+00]]
46
[[ 4.03578560e-01 -4.14462765e-02  1.11102616e+02]
 [ 2.94178318e-02  3.81007309e-01  1.19464602e+02]
 [ 2.23727501e-05 -5.39849566e-05  1.00000000e+00]]
47
[[ 3.93969631e-01 -3.35432180e-02  1.12523418e+02]
 [ 3.26992284e-02  3.78332196e-01  1.18431893e+02]
 [ 2.89362326e-05 -2.30864300e-05  1.00000000e+00]]
36
[[ 3.61440584e-01 -3.50750021e-02  1.33791401e+02]
 [ 2.91500103e-02  3.52860048e-01  1.14074329e+02]
 [ 3.01760559e-06 -2.27058266e-05  1.00000000e+00]]
42
[[ 

28
[[ 2.45790250e-01 -2.07738779e-03  1.27643890e+02]
 [-1.03181189e-02  2.80444548e-01  1.20521521e+02]
 [-7.45810773e-05  1.14972216e-05  1.00000000e+00]]
27
[[ 2.45966173e-01  3.57482118e-03  1.27423834e+02]
 [-1.13035121e-02  2.84194736e-01  1.21705805e+02]
 [-7.87814390e-05  2.52950212e-05  1.00000000e+00]]
18
21
[[ 2.37042379e-01  9.62715740e-03  1.29040693e+02]
 [-1.47036090e-02  2.88556332e-01  1.23057318e+02]
 [-9.82602609e-05  4.65943179e-05  1.00000000e+00]]
21
[[ 2.46959183e-01  5.74600054e-03  1.26240134e+02]
 [-1.02929748e-02  2.86103997e-01  1.23397506e+02]
 [-7.59903644e-05  3.08158239e-05  1.00000000e+00]]
18
25
[[ 2.39655647e-01  1.19331249e-03  1.28190332e+02]
 [-1.34339730e-02  2.82427126e-01  1.25080820e+02]
 [-8.99667971e-05  2.43133551e-05  1.00000000e+00]]
26
[[ 2.43377845e-01  1.28259412e-03  1.26918303e+02]
 [-1.11180531e-02  2.84671109e-01  1.24606173e+02]
 [-8.22212663e-05  2.13941267e-05  1.00000000e+00]]
26
[[ 2.46874889e-01  1.15251019e-03  1.25090923e+02

20
27
[[ 2.46890793e-01 -5.28487988e-02  1.58445952e+02]
 [ 1.82135286e-02  2.55895218e-01  9.32009280e+01]
 [-5.93138792e-05 -4.36163289e-05  1.00000000e+00]]
26
[[ 2.52269103e-01 -5.89957626e-02  1.58109216e+02]
 [ 2.18774942e-02  2.53735987e-01  9.22671078e+01]
 [-4.65865075e-05 -5.73681772e-05  1.00000000e+00]]
23
[[ 2.49297030e-01 -5.59501564e-02  1.57826481e+02]
 [ 2.05474460e-02  2.54202990e-01  9.29088098e+01]
 [-5.34235417e-05 -4.96150321e-05  1.00000000e+00]]
24
[[ 2.56708273e-01 -6.06793662e-02  1.56122428e+02]
 [ 2.33981435e-02  2.52969727e-01  9.17642122e+01]
 [-3.84809680e-05 -6.11087000e-05  1.00000000e+00]]
23
[[ 2.51128926e-01 -5.84726888e-02  1.57576518e+02]
 [ 2.18811105e-02  2.52151331e-01  9.09881558e+01]
 [-4.67996424e-05 -5.66274544e-05  1.00000000e+00]]
19
29
[[ 2.46218041e-01 -5.46144505e-02  1.59459532e+02]
 [ 1.86518115e-02  2.54614783e-01  9.27052964e+01]
 [-5.71538421e-05 -4.84793663e-05  1.00000000e+00]]
17
22
[[ 2.54608698e-01 -5.52922965e-02  1.56017421e

24
[[ 2.40654758e-01  1.85876252e-02  1.25084174e+02]
 [-5.37573783e-02  2.65206625e-01  1.36933214e+02]
 [-7.51459054e-05 -3.29870627e-05  1.00000000e+00]]
25
[[ 2.39628348e-01  1.33371908e-02  1.27646690e+02]
 [-4.93352342e-02  2.64613261e-01  1.35187366e+02]
 [-7.54455256e-05 -3.77800430e-05  1.00000000e+00]]
14
19
22
[[ 2.38399426e-01 -3.96346894e-03  1.31859600e+02]
 [-4.05227720e-02  2.62765011e-01  1.33375422e+02]
 [-8.05979077e-05 -4.44396192e-05  1.00000000e+00]]
19
25
[[ 2.35235763e-01 -1.12362616e-03  1.30316835e+02]
 [-3.92588382e-02  2.64603244e-01  1.32497491e+02]
 [-8.82091323e-05 -3.20028676e-05  1.00000000e+00]]
27
[[ 2.35984384e-01 -8.69723547e-03  1.31318172e+02]
 [-3.47965132e-02  2.60895562e-01  1.30950198e+02]
 [-8.30870104e-05 -4.38317584e-05  1.00000000e+00]]
23
[[ 2.34246763e-01 -1.44703466e-02  1.33237415e+02]
 [-3.10854528e-02  2.59231157e-01  1.29094859e+02]
 [-8.32789891e-05 -4.92702144e-05  1.00000000e+00]]
21
[[ 2.34210875e-01 -1.84976411e-02  1.34598369e

4
7
4
6
4
4
7
6
5
12
4
3
6
4
5
7
9
7
7
6
6
7
11
4
6
8
6
4
8
3
7
2
5
9
5
6
8
11
5
7
8
7
3
6
9
8
7
6
8
3
5
8
7
4
4
5
9
5
11
7
5
4
6
4
4
7
4
5
15
9
7
7
10
4
7
7
9
9
12
5
3
5
7
6
8
4
5
6
6
7
7
8
10
4
7
16
10
12
5
7
2
6
6
8
5
7
7
11
8
8
13
2
6
6
4
3
5
6
5
5
7
5
8
6
7
5
3
10
9
8
6
8
0
8
7
3
9
5
7
7
12
5
7
6
6
3
7
7
8
4
4
5
6
4
1
4
11
5
5
5
6
9
12
11
5
9
6
7
3
9
2
5
8
6
5
4
11
5
7
8
6
9
4
10
7
7
7
7
4
8
5
6
11
7
5
4
9
13
2
5
5
8
8
5
9
4
3
7
12
3
7
4
2
6
8
8
8
6
5
5
6
8
11
4
5
3
4
10
8
6
8
6
11
6
10
7
8
3
4
7
10
8
7
11
8
9
7
4
14
5
6
6
7
5
8
6
3
5
8
4
6
7
7
3
4
7
7
9
6
11
5
6
9
3
5
6
7
5
4
3
2
8
8
5
9
6
6
7
6
12
9
10
5
7
9
4
11
9
6
3
5
6
8
4
9
5
5
4
7
4
8
6
7
5
5
2
4
7
10
10
9
11
7
7
6
7
7
6
6
6
4
4
5
7
3
7
6
2
4
7
10
7
9
10
8
9
5
6
4
4
7
7
5
8
4
6
7
7
6
3
8
8
5
5
11
3
4
6
7
9
8
12
11
7
6
11
7
7
8
12
10
4
5
6
7
4
2
6
9
9
4
5
7
5
7
9
4
9
7
5
8
11
7
10
1
6
9
10
5
3
6
7
6
11
6
11
12
13
4
11
3
6
10
8
8
7
7
7
6
9
2
8
6
5
7
5
13
9
4
10
13
10
8
7
8
4
7
4
10
9
10
9
10
11
6
6
10
3
11
9
11
7
11
6
10
12
